**WEEK 3 - CONTENT BASED COLLABORATIVE FILTERING(CBCF)**

1.**Similarity of movies on the basis of user ratings**

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import time
import os
from os import path
from sklearn.preprocessing import LabelEncoder 
from scipy.linalg import sqrtm

In [2]:
movie = pd.read_csv('/content/movies.csv', sep=',',names=['MovieID','Title','Genres'],encoding='latin-1')
rating = pd.read_csv('ratings.csv',sep=',', names=['UserID','MovieID','Rating','Timestamp'])
user = pd.read_csv('users.csv',sep=',',names=['Gender','Age','Occupation','Zip-code'])

In [ ]:
rating = rating.drop(columns='Timestamp')     #Dropping Timestamp column

In [21]:
#Create dataframe contains movie and rating
movie_rating = pd.merge(movie , rating, on='MovieID')   
movie_rating.head()

,MovieID,Title,Genres,UserID,Rating
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5.0
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4.0
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4.0
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5.0
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5.0


In [22]:
#Tool for cold start problem
#Top high rated movies 
movie_mean=pd.DataFrame(movie_rating.groupby('Title')['Rating'].mean().sort_values(ascending=False))
movie_mean.head(30)

,Rating
Title,
Schlafes Bruder (Brother of Sleep) (1995),5.000000
Smashing Time (1967),5.000000
Follow the Bitch (1998),5.000000
Hour of the Pig,5.000000
Ulysses (Ulisse) (1954),5.000000
Baby,5.000000
Bittersweet Motel (2000),5.000000
Gate of Heavenly Peace,5.000000
I Am Cuba (Soy Cuba/Ya Kuba) (1964),4.750000


In [23]:
#Another tool for cold start problem
#most rated movies
movie_count = pd.DataFrame(movie_rating.groupby('Title')['Rating'].count().sort_values(ascending=False))
movie_count.head(20)

,Rating
Title,
American Beauty (1999),3039
Star Wars: Episode IV - A New Hope (1977),2584
Star Wars: Episode V - The Empire Strikes Back (1980),2572
Star Wars: Episode VI - Return of the Jedi (1983),2498
Jurassic Park (1993),2345
Saving Private Ryan (1998),2324
Terminator 2: Judgment Day (1991),2316
Matrix,2255
Silence of the Lambs,2249


In [24]:
rating_table = movie_rating.pivot_table(index='UserID', columns='Title',values='Rating')
rating_table = rating_table.fillna(0)

In [25]:
rating_table

Title,$1,'Night Mother (1986),'Til There Was You (1997),'burbs,...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),13th Warrior,187 (1997),2 Days in the Valley (1996),20,20 Dates (1998),200 Cigarettes (1999),2001: A Space Odyssey (1968),2010 (1984),24 7: Twenty Four Seven (1997),24-hour Woman (1998),28 Days (2000),3 Ninjas: High Noon On Mega Mountain (1998),3 Strikes (2000),301,39 Steps,400 Blows,42 Up (1998),52 Pick-Up (1986),54 (1998),7th Voyage of Sinbad,8 1/2 (1963),8 1/2 Women (1999),8 Heads in a Duffel Bag (1997),8 Seconds (1994),8MM (1999),A Chef in Love (1996),Abbott and Costello Meet Frankenstein (1948),Abominable Snowman,About Adam (2000),About Last Night... (1986),...,Wonderland (1997),Wonderland (1999),Woo (1998),Wood,Wooden Man's Bride,Working Girl (1988),World Is Not Enough,World of Apu,Wrong Man,Wrong Trousers,Wrongfully Accused (1998),Wyatt Earp (1994),X-Files: Fight the Future,X-Men (2000),X: The Unknown (1956),Xiu Xiu: The Sent-Down Girl (Tian yu) (1998),Yankee Zulu (1994),Yards,Year My Voice Broke,Year of Living Dangerously (1982),Year of the Horse (1997),Yellow Submarine (1968),Yojimbo (1961),You Can't Take It With You (1938),You So Crazy (1994),You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Poisoner's Handbook,Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),Zed & Two Noughts,Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [64]:
 #Using Singular value decomposition
rating_matrix = np.array(rating_table)        #Convert Datafram into matrix
mean = np.mean(rating_matrix, axis=0)             
rating_matrix = rating_matrix + mean              #Changing 0 to higher magnitude values
U, sigma, V=np.linalg.svd(rating_matrix, full_matrices=False)
sigma=np.diag(sigma)
# we take only the k most significant features
sigmat=sqrtm(sigma)           #Change matrix to square root matrix
Usk=np.dot(U,sigma)           #dot product of U and sigma matrices
skV=np.dot(sigma,V)           #dot product of sigma and V matrices
UsV = np.dot(Usk, skV)        #final svd matrices UsV
rating_svd_table = UsV - mean
rating_svd_table = pd.DataFrame(UsV,columns=rating_table.columns,index=rating_table.index)  

In [65]:
rating_svd_table

Title,$1,'Night Mother (1986),'Til There Was You (1997),'burbs,...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),13th Warrior,187 (1997),2 Days in the Valley (1996),20,20 Dates (1998),200 Cigarettes (1999),2001: A Space Odyssey (1968),2010 (1984),24 7: Twenty Four Seven (1997),24-hour Woman (1998),28 Days (2000),3 Ninjas: High Noon On Mega Mountain (1998),3 Strikes (2000),301,39 Steps,400 Blows,42 Up (1998),52 Pick-Up (1986),54 (1998),7th Voyage of Sinbad,8 1/2 (1963),8 1/2 Women (1999),8 Heads in a Duffel Bag (1997),8 Seconds (1994),8MM (1999),A Chef in Love (1996),Abbott and Costello Meet Frankenstein (1948),Abominable Snowman,About Adam (2000),About Last Night... (1986),...,Wonderland (1997),Wonderland (1999),Woo (1998),Wood,Wooden Man's Bride,Working Girl (1988),World Is Not Enough,World of Apu,Wrong Man,Wrong Trousers,Wrongfully Accused (1998),Wyatt Earp (1994),X-Files: Fight the Future,X-Men (2000),X: The Unknown (1956),Xiu Xiu: The Sent-Down Girl (Tian yu) (1998),Yankee Zulu (1994),Yards,Year My Voice Broke,Year of Living Dangerously (1982),Year of the Horse (1997),Yellow Submarine (1968),Yojimbo (1961),You Can't Take It With You (1938),You So Crazy (1994),You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Poisoner's Handbook,Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),Zed & Two Noughts,Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,95.084641,215.184433,109.761281,681.891137,516.273712,4.539584,1682.563341,1771.082551,891.565926,2020.583414,1541.044671,96.356688,684.354827,1572.633909,249.022044,381.467444,4784.840452,1123.116783,14.708225,14.171167,1106.826874,58.639789,8.495347,22.439146,711.860763,532.597560,251.076294,318.709374,498.162053,661.095610,631.815494,81.912672,234.222212,200.390788,574.780730,58.483035,560.471234,12.686948,40.518316,746.215218,...,45.576975,181.607252,14.471402,211.041602,3.017935,1333.118316,1970.788238,142.006201,214.086779,2982.533004,245.467167,627.778144,2419.493165,4235.523550,26.275539,157.966204,2.038761,182.290327,88.525245,1084.448602,11.885393,1211.165075,620.754345,228.702005,37.797547,2099.732589,166.572444,3617.640081,1385.642968,773.871991,217.633730,930.373307,38.072984,263.728735,4.676133,65.282218,867.869083,6.449996,58.793892,900.303321
2,109.613380,243.134512,132.603817,841.741187,749.464643,4.032910,2077.395243,1773.057206,1004.744687,2648.628049,2221.405840,140.916248,931.027511,2018.963076,344.983136,452.346441,6218.095156,1480.653588,23.022549,20.650803,1381.843246,62.549135,10.607017,25.763681,946.100419,729.175887,363.974164,485.865404,642.876683,902.662011,853.299604,110.963615,306.759064,267.411984,772.078213,82.685992,721.161933,19.664949,47.041897,951.476810,...,46.065049,231.546812,18.883591,259.969831,5.667725,1717.598655,2837.721787,206.738656,318.181981,3309.028042,315.327303,940.338269,3222.398760,5509.488542,35.967586,257.235420,4.425182,239.110710,94.178983,1561.650517,19.536279,1328.315908,1053.166390,298.473824,31.868687,2712.486788,208.211972,4434.050045,1960.469373,1143.713448,276.022730,1258.132242,42.545694,343.396693,10.106904,95.212291,1101.639444,11.247462,66.749546,1174.026717
3,105.620783,162.991846,89.619535,726.050186,527.362300,2.594828,1658.564536,1489.310336,812.397725,1939.453999,1732.564198,97.343692,696.077617,1577.597131,274.325124,359.758096,4825.142127,1114.930958,15.202925,11.870019,1292.660276,53.607733,10.081311,27.661576,693.522044,519.297143,262.186754,376.905228,489.130535,714.815003,626.653779,89.124820,267.914684,196.063453,581.497722,63.777894,575.889867,14.579231,34.262619,789.002289,...,41.784221,167.587635,21.160764,189.375192,4.573149,1338.746736,2094.557

In [74]:
#movies recommendation function 
def recommend(movie_name, n):
  movies = rating_svd_table[movie_name] 
  similar_movies = rating_svd_table.corrwith(movies)
  related_movie = pd.DataFrame(similar_movies , columns=['Correlation'])
  related_movie = related_movie['Correlation'].sort_values(ascending=False)
  related_movie = pd.DataFrame(related_movie)
  print('Hello sir, There are top {}  movies with similar ratings of movie {} \n hope you would like them'.format(n,movie_name))
  return related_movie.head(n)

In [75]:
recommend('Shawshank Redemption',10)

Hello sir, There are top 10  movies with similar ratings of movie Shawshank Redemption 
 hope you would like them


,Correlation
Title,
Shawshank Redemption,1.000000
Silence of the Lambs,0.996625
Schindler's List (1993),0.996153
Pulp Fiction (1994),0.995275
Good Will Hunting (1997),0.994847
Fargo (1996),0.994594
Usual Suspects,0.993078
American Beauty (1999),0.992837
Sling Blade (1996),0.992488


2. **Similarity of movies on the basis of Genre**

In [28]:
le = LabelEncoder()
movie_rating['enc_Genres'] = le.fit_transform(movie_rating['Genres'])

In [29]:
len(movie_rating['enc_Genres'].unique())

550

In [30]:
#Create new pivot table with values as 'Genre'
genre_table = movie_rating.pivot_table(values='enc_Genres',index='UserID',columns='Title')
genre_table = genre_table.fillna(0)

In [52]:
genre_table

Title,$1,'Night Mother (1986),'Til There Was You (1997),'burbs,...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),13th Warrior,187 (1997),2 Days in the Valley (1996),20,20 Dates (1998),200 Cigarettes (1999),2001: A Space Odyssey (1968),2010 (1984),24 7: Twenty Four Seven (1997),24-hour Woman (1998),28 Days (2000),3 Ninjas: High Noon On Mega Mountain (1998),3 Strikes (2000),301,39 Steps,400 Blows,42 Up (1998),52 Pick-Up (1986),54 (1998),7th Voyage of Sinbad,8 1/2 (1963),8 1/2 Women (1999),8 Heads in a Duffel Bag (1997),8 Seconds (1994),8MM (1999),A Chef in Love (1996),Abbott and Costello Meet Frankenstein (1948),Abominable Snowman,About Adam (2000),About Last Night... (1986),...,Wonderland (1997),Wonderland (1999),Woo (1998),Wood,Wooden Man's Bride,Working Girl (1988),World Is Not Enough,World of Apu,Wrong Man,Wrong Trousers,Wrongfully Accused (1998),Wyatt Earp (1994),X-Files: Fight the Future,X-Men (2000),X: The Unknown (1956),Xiu Xiu: The Sent-Down Girl (Tian yu) (1998),Yankee Zulu (1994),Yards,Year My Voice Broke,Year of Living Dangerously (1982),Year of the Horse (1997),Yellow Submarine (1968),Yojimbo (1961),You Can't Take It With You (1938),You So Crazy (1994),You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Poisoner's Handbook,Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),Zed & Two Noughts,Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,451.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,336.0,336.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193.0,363.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [32]:
 #Making dictionary to store movie titles with indices
 movie_titles= {
     movie : i for i, movie in
     enumerate(list(genre_table.columns))
 }

In [17]:
!pip install fuzzywuzzy

In [18]:
#Fuzz for comparison between two strings
from fuzzywuzzy import fuzz                   

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [57]:
def recommend(movie_name, n):
  movies = genre_table[movie_name]
  similar_movies = rating_table.corrwith(movies)
  related_movie = pd.DataFrame(similar_movies , columns=['Correlation'])
  related_movie = related_movie['Correlation'].sort_values(ascending=False)
  genre = list(movie_rating[(movie_rating['Title'] ==movie_name)]['Genres'].unique())
  related_movie = pd.DataFrame(related_movie)
  print('Hello sir, There are top {} {}  movies for you'.format(n ,genre ))
  return related_movie.head(n) 

In [58]:
recommend('Star Trek VI: The Undiscovered Country (1991)',16)

Hello sir, There are top 16 ['Action|Adventure|Sci-Fi']  movies for you


,Correlation
Title,
Star Trek VI: The Undiscovered Country (1991),0.949171
Star Trek: Generations (1994),0.696806
Star Trek: First Contact (1996),0.677540
Star Trek III: The Search for Spock (1984),0.612958
Star Trek IV: The Voyage Home (1986),0.608852
Star Trek: Insurrection (1998),0.608262
Star Trek: The Wrath of Khan (1982),0.557496
Star Trek V: The Final Frontier (1989),0.552530
Stargate (1994),0.518119
